In [1]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess
from twisted.internet import reactor
import scrapy
from scrapy.crawler import CrawlerRunner
from scrapy.utils.log import configure_logging
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('quoteresult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [2]:
import json
with open('harryPotterCharacters.json') as f:
    characters = [x.lower() for x in json.load(f)["characters"]]

In [3]:
import re
from datetime import datetime
from datetime import timedelta
from dateutil import parser

def isUserLink(href):
    m = re.match('/u/\d+/\w+', href)
    if(m is None):
        return False
    return True
def isStoryLink(href):
    m = re.match('.*(?P<link>/s/\d+(/\d+)?/.*)', href)
    if(m is None):
        return False
    return True
def extractStoryLink(href):
    m = re.match('.*(?P<link>/s/\d+(/\d+)?/.*)', href)
    if(m is None):
        return ""
    return m.group('link')
def isReviewLink(href):
    m = re.match('/r/\d+', href)
    if(m is None):
        return False
    return True
def convertDate(strDate):
    now = datetime.now()
    mSeconds = re.match('(?P<seconds>\d+)s', strDate)
    mMinutes = re.match('(?P<minutes>\d+)m', strDate)
    mHours = re.match('(?P<hours>\d+)h', strDate)
    
    mDate = re.match('(?P<month>\w{3}) (?P<day>\d+)(, (?P<year>\d{4}))?', strDate)
    delta = timedelta(0)
    if(mSeconds is not None):
        delta = timedelta(0,int(mSeconds.group('seconds')))
    if(mMinutes is not None):
        delta = timedelta(minutes = int(mMinutes.group('minutes')))  
    if(mHours is not None):
        delta = timedelta(hours = int(mHours.group('hours'))) 
    if(mDate is not None):
        return parser.parse(strDate)
    return now + delta

def getOtherInfoAsJson(other_stuff):
    language = ''
    genre = ''
    favorites = 0
    follows = 0
    reviews = 0
    words = -1
    chapters = 1
    language_genre_match = re.search(
            '(?P<language>\w+) - ((?P<genre1>\w+)/(?P<genre2>\w+))', 
            other_stuff)
    if(language_genre_match is not None):
        language = language_genre_match.group('language')
        genre = [language_genre_match.group('genre1'), language_genre_match.group('genre2')]
    
    mReviews = re.search(
            'Reviews: <a.*>(?P<reviews>\d+)</a>', 
            other_stuff)
    if(mReviews is not None):
        reviews = mReviews.group('reviews')
        
    mWords = re.search(
            'Words: (?P<words>(\d+,?)+)', 
            other_stuff)
    if(mWords is not None):
        words = mWords.group('words')
    
    mChapters = re.search(
            'Chapters: (?P<chapters>(\d+,?)+)', 
            other_stuff)
    if(mChapters is not None):
        chapters = mChapters.group('chapters')
    
    mFavorites = re.search(
            'Favs: (?P<favorites>\d+)', 
            other_stuff)
    if(mFavorites is not None):
        favorites = mFavorites.group('favorites')

    mFollows = re.search(
            'Follows: (?P<follows>\d+)', 
            other_stuff)
    if(mFollows is not None):
        follows = mFollows.group('follows')
        
    return {
                'language':language,
                'genre': genre,
                'favorites': favorites,
                'follows': follows,
                'reviews': reviews,
                'words': words,
                'chapters': chapters
           }

In [4]:
import logging
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string
sid = SentimentIntensityAnalyzer()

class FanFicSpider(scrapy.Spider):
    name = "fanFic"
    start_urls = [
        'https://www.fanfiction.net/s/13025005/1/A-Twist-In-Time'
        #'https://www.fanfiction.net/s/13090372/1/This-Labyrinth-of-Suffering'
        #'https://www.fanfiction.net/s/13059900/1/La-For%C3%AAt-des-%C3%82mes-Bris%C3%A9es',
        #'https://www.fanfiction.net/r/8636004/'
    ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'fanficResult.json'                        # Used for pipeline 2
    }
       
    def parseUserPage(self, response):
        profile_top = response.xpath('//div[@id="content_wrapper_inner"]')
        
        #follow links to their stories and reviews
        for elem in response.xpath('//div[@class="z-list mystories"]').xpath(".//@href"):
            if(isStoryLink(elem.extract())):
                yield response.follow(elem.extract(), callback=self.parse)
                
            if(isReviewLink(elem.extract())):
                yield response.follow(elem.extract(), callback=self.parseReview)
        
        #get the favorites
        favorites = []
        for elem in response.xpath('//div[@class="z-list favstories"]'):
            favStory = '';
            favAuthor = ''
            for link in elem.xpath('./a//@href').extract():
                if(isUserLink(link)):
                    favAuthor = link
                    yield response.follow(link, callback=self.parseUserPage)
                elif(isStoryLink(link)):
                    favStory = link
                    yield response.follow(link, callback=self.parse)
                elif(isReviewLink(link)):
                    yield response.follow(link, callback=self.parseReview)
            favorites.append({
                'favStory' : favStory,
                'favAuthor': favAuthor
            })
        
        yield {
            'pageType': 'user',
            'name': profile_top.xpath('.//span/text()').extract_first(),
            'favorites': favorites
        }
        
    def parse(self, response):
        profile_top = response.xpath('//div[@id="profile_top"]')
        storyName = response.xpath('//link[@rel="canonical"]//@href').extract_first()
        storyName = extractStoryLink(storyName)
        
        abstract = (profile_top.xpath('.//div/text()').extract_first())
        rating = profile_top.xpath('.//span/a/text()').extract_first()
        otherInfo = profile_top.xpath('.//span').extract()[3]
        
        date = convertDate(profile_top.xpath('.//span/text()').extract()[3])
        storyType = response.xpath('//div[@id="pre_story_links"]').xpath('.//a/@href').extract()[-1]
        text = response.xpath('//div[@id="storytext"]').extract_first().lower()
        
        #find relevant characters
        characterFreq = {}
        for character in characters:
            if(character in text):
                if(character not in characterFreq):
                    characterFreq[character] = 0
                characterFreq[character] += text.count(character)
          
        #get author
        author = ''
        for link in profile_top.xpath('.//@href'):
            if(isUserLink(link.extract())):
                author = link.extract()
                yield response.follow(author, callback=self.parseUserPage)
        
        yield {
            'pageType': 'story',
            'storyLink': storyName,
            'author': author,
            'title': response.xpath('//title/text()').extract_first(),
            'storyType': storyType,
            'abstract': abstract,
            'rating': rating,
            'otherInfo': getOtherInfoAsJson(otherInfo),
            'date': date.strftime('%Y-%m-%d %M:%S'),
            'characters': characterFreq
        }

    def parseReview(self, response):
        reviewOf = response.xpath('//th').xpath('.//@href').extract_first()
        for review in response.xpath('//table[@id="gui_table1i"]//td'):
            reviewBody = (review.xpath('.//div/text()').extract_first())
            #remove punctuation
            table = str.maketrans("", "", string.punctuation)
            reviewBody = reviewBody.translate(table)
            sentimentScore = sum([sid.polarity_scores(word)['compound'] for word in reviewBody.split()])
            reviewer = (review.xpath('./text()').extract_first())
            if(reviewer is ' '):
                reviewer = (review.xpath('./a/@href').extract_first())
                yield response.follow(reviewer, callback=self.parseUserPage)
            yield{
                'pageType': 'review',
                'reviewOf': reviewOf,
                'reviewer': reviewer,
                'reviewBody': reviewBody,
                'sentimentScore': sentimentScore
            }
            
from crochet import setup
setup()
configure_logging({'LOG_FORMAT': '%(levelname)s: %(message)s'})
def run_spider():
    runner = CrawlerRunner()
    d = runner.crawl(FanFicSpider)

run_spider()

C:\Users\ryanc\Anaconda3\envs\cs470\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'fanficResult.json', 'LOG_LEVEL': 30}
2018-10-16 14:43:15 [scrapy.core.downloader.handlers.http11] WARNING: Got data loss in https://www.fanfiction.net/u/1465660/Ravenclaw-Slytherin. If you want to process broken responses set the setting DOWNLOAD_FAIL_ON_DATALOSS = False -- This message won't be shown in further requests


<span class="xgray xcontrast_txt">Rated: <a class="xcontrast_txt" href="https://www.fictionratings.com/" target="rating">Fiction  T</a> - English - Mystery/Adventure - Chapters: 13   - Words: 1,123,123,40,782 - Reviews: <a href="/r/13025005/">82</a> - Favs: 13 - Follows: 19 - Updated: <span data-xutime="1539713210">53m</span> - Published: <span data-xutime="1533456146">8/5</span> - id: 13025005 </span>


{'language': 'English',
 'genre': ['Mystery', 'Adventure'],
 'favorites': '13',
 'follows': '19',
 'reviews': '82',
 'words': '1,123,123,40,782',
 'chapters': '13'}